In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import time
import shap
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_validate, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix, classification_report
import xgboost as xgb
from pandas_profiling import ProfileReport

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

In [3]:
df_player = pd.read_csv("data/Players.csv")
df_player_stats = pd.read_csv("data/player_data.csv")
df_player_stats.loc[df_player_stats.name == "Mike James", "year_start"] = 2002
df_player_stats.loc[df_player_stats.name == "Mike James", "year_end"] = 2014
df_player_stats = df_player_stats.loc[df_player_stats.year_start < 2018].copy()
df_player_stats = df_player_stats.loc[~(df_player_stats.name == "Jo Jo English")].copy()
df_player_stats["birth_date"] = pd.to_datetime(df_player_stats["birth_date"], infer_datetime_format = True)
df_season = pd.read_csv("data/Seasons_Stats.csv")
df_season["name"] = df_season.Player.str.replace("*","")
df_season = df_season.loc[~df_season.name.isnull()].copy()
df_season.loc[((df_season.Year > 1990)&(df_season.name.str.contains("John Lucas"))), "name"] = "John Lucas III"
df_season.loc[df_season.name == "Sheldon McClellan", "name"] = "Sheldon Mac"
df_season.loc[df_season.name == "Johnny Macknowski", "name"] = "Johnny Macknowsky"
df_season.loc[df_season.name == "Walter Tavares", "name"] = "Edy Tavares"
df_season.drop(columns = ["Unnamed: 0","blanl","blank2"], inplace = True)

In [3]:
name_map = {'Billy Ray': 'Billy Ray Bates',
 'Butch Van': 'Butch Van Breda Kolff',
 'Dick Van': 'Dick Van Arsdale',
 'Eddie Lee': 'Eddie Lee Wilkins',
 'George Bon': 'George Bon Salle',
 'Horacio Llamas': 'Horacio Llamas Grey',
 'Hot Rod': 'Hot Rod Williams',
 'James Michael': 'James Michael McAdoo',
 'Jan Van': 'Jan Van Breda Kolff',
 'Jo Jo': 'Jo Jo White',
 'Joe Barry': 'Joe Barry Carroll',
 'Juan Carlos': 'Juan Carlos Navarro',
 'Keith Van': 'Keith Van Horn',
 'Logan Vander': 'Logan Vander Velden',
 'Luc Mbah': 'Luc Mbah a Moute',
 'Metta World': 'Metta World Peace',
 'Micheal Ray': 'Micheal Ray Richardson',
 'Nando De': 'Nando De Colo',
 'Nick Van': 'Nick Van Exel',
 'Norm Van': 'Norm Van Lier',
 'Peter John': 'Peter John Ramos',
 'Tom Van': 'Tom Van Arsdale',
 'Vinny Del': 'Vinny Del Negro',
 'Wah Wah': 'Wah Wah Jones',
 'Whitey Von': 'Whitey Von Nieda',
 'World B.': 'World B. Free'}
for key, val in name_map.items():
    df_season.loc[df_season.name == key, "name"] = val

In [4]:
df_season.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,name
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,NaN,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,3.6,3.5,NaN,NaN,NaN,NaN,NaN,144.0,516.0,0.279,NaN,NaN,NaN,144.0,516.0,0.279,0.279,170.0,241.0,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0,Curly Armstrong
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,NaN,0.387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.6,2.2,NaN,NaN,NaN,NaN,NaN,102.0,274.0,0.372,NaN,NaN,NaN,102.0,274.0,0.372,0.372,75.0,106.0,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0,Cliff Barker
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.8,3.6,NaN,NaN,NaN,NaN,NaN,174.0,499.0,0.349,NaN,NaN,NaN,174.0,499.0,0.349,0.349,90.0,129.0,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0,Leo Barnhorst
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,NaN,0.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,22.0,86.0,0.256,NaN,NaN,NaN,22.0,86.0,0.256,0.256,19.0,34.0,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0,Ed Bartels
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,NaN,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,21.0,82.0,0.256,NaN,NaN,NaN,21.0,82.0,0.256,0.256,17.0,31.0,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0,Ed Bartels


In [13]:
df_player_stats.birth_date.dt.year

0       1968.0
1       1946.0
2       1947.0
3       1969.0
4       1974.0
         ...  
4544    1994.0
4546    1953.0
4547    1948.0
4548    1997.0
4549    1919.0
Name: birth_date, Length: 4460, dtype: float64

In [14]:
df_season.Year - df_season.Age

0        1919.0
1        1921.0
2        1925.0
3        1926.0
4        1926.0
          ...  
24686    1993.0
24687    1990.0
24688    1997.0
24689    1995.0
24690    1998.0
Length: 24624, dtype: float64

In [10]:
df_player_stats.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,1968-06-24,Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,1946-04-07,Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,1947-04-16,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,1969-03-09,Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,1974-11-03,San Jose State University


In [18]:
df = df_season.merge(df_player_stats, on = "name", how = "left")

In [19]:
#profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

In [20]:
df.describe()

,Year,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,year_start,year_end,weight
count,25276.000000,25268.000000,25276.000000,18749.000000,24787.000000,24750.000000,25188.000000,19374.000000,25174.000000,21385.000000,21385.000000,22191.000000,23195.000000,21385.000000,21385.000000,20134.000000,20193.000000,25237.000000,25237.000000,25237.000000,24750.000000,21390.000000,21390.000000,21390.000000,21390.000000,25276.000000,25276.000000,25174.000000,19463.000000,19463.000000,15874.000000,25276.000000,25276.000000,25141.000000,25174.000000,25276.000000,25276.000000,24392.000000,21390.000000,21390.000000,24961.000000,25276.000000,21390.000000,21390.000000,20198.000000,25276.000000,25276.000000,25276.000000,25276.000000,25271.000000
mean,1992.588424,26.684304,50.826317,23.689743,1211.144632,12.481665,0.493251,0.158975,0.325273,6.165022,13.685434,9.928899,13.036504,1.648819,1.420215,15.081102,18.920423,1.260031,1.229972,2.491073,0.065042,-1.770645,-0.543609,-2.314030,0.565783,195.744382,431.222029,0.431071,22.244104,63.719622,0.248663,178.615999,382.156710,0.445710,0.450969,102.319710,136.615089,0.719503,62.221692,147.363534,224.479989,115.188044,40.010472,24.770126,74.207595,116.460753,510.936857,1988.432663,1996.697618,212.075739
std,17.340278,3.854205,26.527986,28.698872,942.488797,6.038687,0.094420,0.187875,0.221787,4.853608,6.643834,5.041705,9.219115,1.013455,1.797625,6.911416,5.471004,2.136701,1.272058,3.059990,0.102432,3.792329,2.271906,4.697556,1.339261,188.702060,398.506021,0.095937,38.558287,102.510986,0.176751,179.954455,371.760447,0.099736,0.099133,113.162145,145.764933,0.141772,67.289971,146.086890,227.865287,136.141665,38.794079,37.678685,67.923189,84.940412,494.048109,17.133375,17.774624,26.434199
min,1950.000000,18.000000,1.000000,0.000000,0.000000,-90.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.100000,-1.000000,-2.800000,-2.519000,-73.800000,-30.400000,-86.700000,-2.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1947.000000,1947.000000,133.000000
25%,1981.000000,24.000000,27.000000,0.000000,339.000000,9.800000,0.458000,0.005000,0.208000,2.600000,8.700000,5.900000,6.500000,1.100000,0.300000,11.300000,15.400000,-0.100000,0.200000,0.200000,0.031000,-3.400000,-1.700000,-4.200000,-0.200000,41.000000,98.000000,0.393000,0.000000,1.000000,0.100000,35.000000,82.000000,0.407000,0.414000,18.000000,27.000000,0.658000,12.000000,33.000000,51.000000,19.000000,9.000000,3.000000,18.000000,39.000000,106.000000,1977.000000,1985.000000,190.000000
50%,1996.000000,26.000000,58.000000,8.000000,1055.000000,12.700000,0.506000,0.064000,0.295000,5.400000,12.700000,9.200000,10.500000,1.500000,0.900000,14.200000,18.700000,0.400000,0.800000,1.400000,0.075000,-1.500000,-0.500000,-1.800000,0.000000,141.000000,321.000000,0.439000,3.000000,12.000000,0.292000,122.000000,270.000000,0.456000,0.463000,63.000000,88.000000,0.743000,38.000000,106.000000,159.000000,68.000000,29.000000,11.000000,56.000000,109.000000,364.000000,1991.000000,2000.000000,210.000000
75%,2007.000000,29.000000,76.000000,46.000000,1973.000000,15.600000,0.544000,0.289000,0.400000,9.000000,18.100000,13.500000,17.600000,2.100000,1.900000,17.700000,22.200000,2.000000,1.800000,3.800000,0.115000,0.300000,0.700000,0.300000,0.900000,299.250000,663.000000,0.480000,27.000000,84.000000,0.363000,269.000000,581.000000,0.497000,0.501000,149.000000,201.000000,0.808000,91.000000,213.000000,322.000000,161.000000,60.000000,30.000000,113.000000,183.000000,780.000000,2002.000000,2012.000000,230.000000
max,2017.000000,44.000000,88.000000,83.000000,3882.000000,129.100000,1.136000,1.000000,6.000000,100.000000,100.000000,100.000000,100.000000,24.2000

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25276 entries, 0 to 25275
Data columns (total 58 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        25276 non-null  float64
 1   Player      25276 non-null  object 
 2   Pos         25276 non-null  object 
 3   Age         25268 non-null  float64
 4   Tm          25276 non-null  object 
 5   G           25276 non-null  float64
 6   GS          18749 non-null  float64
 7   MP          24787 non-null  float64
 8   PER         24750 non-null  float64
 9   TS%         25188 non-null  float64
 10  3PAr        19374 non-null  float64
 11  FTr         25174 non-null  float64
 12  ORB%        21385 non-null  float64
 13  DRB%        21385 non-null  float64
 14  TRB%        22191 non-null  float64
 15  AST%        23195 non-null  float64
 16  STL%        21385 non-null  float64
 17  BLK%        21385 non-null  float64
 18  TOV%        20134 non-null  float64
 19  USG%        20193 non-nul

In [22]:
df.to_csv("data/nba.csv", index= False)

# appendix

## initial assumption is that due to the only non merged players being prior to 1977, the ABA NBA merger, all remaining 491 Players played in non NBA leagues (ABA, canadian league, semi pro, etc) , with at least 161 of them being from the ABA, 

In [11]:
df_test = df_season.merge(df_player_stats, on = "name", how = "outer",indicator = True)


In [12]:
df_test._merge.value_counts()

both          25276
right_only      491
left_only         0
Name: _merge, dtype: int64

In [207]:
m1 = df.year_start < 2018
m2 = df.year_start > 1949
m3 = m1 & m2
df.loc[m3]._merge.value_counts()

both          24874
right_only      304
left_only         0
Name: _merge, dtype: int64

In [230]:
df.loc[df._merge == "right_only"].groupby("year_start")._merge.value_counts().sort_values(ascending = False)

year_start  _merge    
1947        right_only    123
1968        right_only    114
1969        right_only     46
1949        right_only     40
1971        right_only     34
1970        right_only     31
1948        right_only     24
1972        right_only     19
1973        right_only     19
1974        right_only     14
1975        right_only     14
1976        right_only     12
1958        right_only      1
Name: _merge, dtype: int64

In [231]:
df.loc[df._merge == "right_only"].groupby("year_end")._merge.value_counts().sort_values(ascending = False)

year_end  _merge    
1947      right_only    85
1949      right_only    71
1968      right_only    55
1970      right_only    43
1969      right_only    40
1975      right_only    32
1948      right_only    31
1976      right_only    31
1972      right_only    30
1971      right_only    29
1973      right_only    28
1974      right_only    15
1963      right_only     1
Name: _merge, dtype: int64

In [228]:
right_names = df.loc[((df._merge == "right_only"))].name.unique()
right_names.sort()
right_names

array(['Al Beard', 'Al Brightman', 'Al Cueto', 'Al Lucas', 'Al Lujack',
       'Al Negratti', 'Al Salvadori', 'Al Sanders', 'Al Smith',
       'Al Williams', 'Andrew Anderson', 'Andy Kostecka',
       'Andy Tonkovich', 'Angelo Musi', 'Ansley Truitt', 'Armand Cure',
       'Art Hillhouse', 'Art Stolkey', 'Arthur Becker', 'Arvesta Kelly',
       'Aubrey Davis', 'Aud Brindley', 'Aulcie Perry', 'Barry Liebowitz',
       'Barry Parkhill', 'Ben Goldfaden', 'Ben Schadler', 'Ben Scharnus',
       'Bill Allen', 'Bill Bunting', 'Bill Crow', 'Bill Davis',
       'Bill Downey', 'Bill Gaines', 'Bill Garner', 'Bill Higgins',
       'Bill Keller', 'Bill Meyer', 'Bill Miller', 'Bill Newton',
       'Billy DeAngelis', 'Billy Evans', 'Billy Harris', 'Billy James',
       'Billy Schaeffer', 'Billy Shepherd', 'Bob Bedell', 'Bob Cluggish',
       'Bob Dille', 'Bob Faught', 'Bob Fitzgerald', 'Bob Ford',
       'Bob Gantt', 'Bob Hubbard', 'Bob Lackey', 'Bob McIntyre',
       'Bob Mullens', 'Bob Netolicky', "

In [215]:
else_names = df.loc[~((df._merge == "right_only"))].name.unique()

In [224]:
def editDistDP(str1, str2, m, n):
    # Create a table to store results of subproblems
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
 
    # Fill d[][] in bottom up manner
    for i in range(m + 1):
        for j in range(n + 1):
 
            # If first string is empty, only option is to
            # insert all characters of second string
            if i == 0:
                dp[i][j] = j    # Min. operations = j
 
            # If second string is empty, only option is to
            # remove all characters of second string
            elif j == 0:
                dp[i][j] = i    # Min. operations = i
 
            # If last characters are same, ignore last char
            # and recur for remaining string
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
 
            # If last character are different, consider all
            # possibilities and find minimum
            else:
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert
                                   dp[i-1][j],        # Remove
                                   dp[i-1][j-1])    # Replace
 
    return dp[m][n]

def similar_names(right_names, else_names):
    similar_names = []
    for rname in right_names:
        for lname in else_names:
            cur_edit = editDistDP(rname, lname, len(rname), len(lname))
            if cur_edit < 3:
                similar_names.append((rname,lname))
    return similar_names

In [225]:
right_names = df.loc[((df._merge == "right_only"))].name.unique()
else_names = df.loc[~((df._merge == "right_only"))].name.unique()
sim_names = similar_names(right_names, else_names)

In [226]:
sim_names

[('Johnny Bach', 'Johnny Baum'),
 ('Norm Baker', 'Norm Mager'),
 ('John Barr', 'John Barber'),
 ('John Barr', 'Jon Barry'),
 ('Charles Beasley', 'Charles Bradley'),
 ('John Beasley', 'John Bagley'),
 ('Jim Bradley', 'Joe Bradley'),
 ('Larry Bunce', 'Larry Nance'),
 ('Bill Bunting', 'Bill Buntin'),
 ('Walter Byrd', 'Walter Bond'),
 ('Don Carlos', 'Don Carlson'),
 ('Joe Colone', 'Joe Cooke'),
 ('Larry Conley', 'Larry Comley'),
 ('Lee Davis', 'Red Davis'),
 ('Lee Davis', 'Mel Davis'),
 ('Lee Davis', 'Ben Davis'),
 ('Don Dee', 'Ron Lee'),
 ('Bob Dille', 'Bob Doll'),
 ('Bob Dille', 'Bob Miller'),
 ('Bob Ford', 'Bob Wood'),
 ('Bob Ford', 'Don Ford'),
 ('Donnie Forman', 'Connie Norman'),
 ('Bill Garner', 'Bill Turner'),
 ('Bill Garner', 'Bill Garnett'),
 ('Joe Hamilton', 'Roy Hamilton'),
 ('Dennis Holman', 'Dennis Rodman'),
 ('Mike Jackson', 'Luke Jackson'),
 ('Jim Jarvis', 'Jim Davis'),
 ('Jim Jarvis', 'Jim Garvin'),
 ('Stew Johnson', 'Steve Johnson'),
 ('Gary Keller', 'Gary Zeller'),
 ('Ron

In [249]:
right_names = df.loc[((df._merge == "right_only"))].name.unique()


In [250]:
rev_right_names = (pd.Series(right_names).str.split(" ").apply(lambda x: x[::-1])).apply(lambda x : ", ".join(x)).values

In [248]:
ABA = """A
Adams, Don
Adams, George
Aitch, Matthew
Akin, Henry
Alexander, Clyde
Allen, Bill
Allen, Willie
Aloi, NIck
Altengarten, Jim	Anderson, Andy
Anderson, Cliff
Anderson, Dan
Anderson, Willie
Andrews, Jim
Angel, David
Ard, Jim
Armstrong, Warren	Arnzen, Bob
Artis, Orsten
Arwood, Vic
Ashley, Carl
Atkins, Bill
Austin, John
Averitt, Bird
B
Bacon, Henry
Baker, Jimmie
Bailey, Bill
Banks, Charley
Banks, Walker
Barnes, Marvin
Barnett,Nate
Barnett, Steve
Barnhill, John
Barr, Mike
Barrett, Mike
Barry, Rick	Bassett, Tim
Batts, Lloyd
Baum, John
Bayne, Howard
Beard, Al
Beasley, Charles
Beasley, John
Beaty, Zelmo
Beck, Byron
Becker, Art
Bedell, Bob
Beechum, Earl
Behrens, Rod
Belcher, Bob
Benfield, Bob	Bennett, Dave
Bennett, Mel
Bennett, Spider
Benson, Andy
Benson, Vern
Berg, Larry
Bethea, Jo Jo
Bialosuknia, Wes
Billingy, Lionel
Bissant, Bob
Black, Tom
Blalock, Roger
Blanchard, Bobby
Blondet, Hector	Blue, Pat
Bobik, Ralph
Bodell, Bob
Bogash, Gene
Bogoyevac, Ron
Bohnenstiehl, Rodger
Bonaparte, Charley
Bonham, Ron
Booker, Hal
Boone, Ron
Bosak, Dick
Bowens, Tom
Bowling, Orbie
Bowman, Nate
Boylan, Mike
Bradds, Gary
Bradley, Bill
Bradley, Jim
Branch, Nate
Branson, Jess
Brent, David
Brewster, DeWayne
Brisker, John
Bristow, Allan
Brittelle, Doug
Brookins, Clarence
Brown, Larry
Brown, Matt
Brown, Roger
Brown, Roger
Brown, Willie
Bruce, Kirk
Brunkhorst, Brian
Bruns, George
Bryant, George
Bucci, George
Buford, Billy
Bunce, Larry
Buntin, Bill	Bunting, Bill
Bunton, Bill
Burden, Ticky
Burlong, Henry
Burney, Clarence
Burns, Jim
Burrell, Biff
Burwell, Bill
Buse, Don
Bustion, Dave
Butler, Bill
Butler, Mike
Byers, Tony
Byrd, Walt
C
Calabria, Chad
Caldwell, Jim
Caldwell, Joe
Calvin, Mack
Campbell, Willie
Cannon, Larry
Card, Frank
Carlos, Don
Carney, Greg	Carr, M.L
Carrier, Darel
Carter, George
Casey, Mike
Cash, Cornelius
Cash, Sam
Chamberlain, Bill
Chambers, Jerry
Champion, Tom
Chaney, Don	Chapman, Clint
Chapman, Wayne
Chappell, Len
Cherry, Willie
Chesnut, Tom
Childress, Mike
Chimielewski, Bill
Chones, Jim
Christian, Bob
Chubin, Steve	Cisson, Greg
Clark, Dick
Clark, George
Clawson, John
Clifford, Dennis
Cluess, Greg
Cluess, Hank
Clyde, Benny
Cobb, Ric
Collins, John
Combs, Glen
Comeaux, John
Congden, Jeff
Conley, Larry
Conner, Jimmy Dan
Contreras, Mike
Conway, Lyndall
Cooke, Joe
Cooper, Royce
Copeland, Bernie
Core, C.A.
Cornwall, Richie
Costello, Fran
Coulter, Leonard
Cramer.Steve
Creighton, Jim
Crenshaw, Dan
Critchfield, Russ
Crocker, Jerry
Croft, Bobby
Cromer, Joe
Cronk, Ray
Crow, Bill
Crowell, Ron
Crowell, Steve
Crump, Art
Cueto, Al
Cunningham, Billy
Curnutt, Don
Cuyler, Sylvester
D
Dabich, Mike
Dampier, Louie
Daniels, Mel
Daniels, Reuben
D'Antoni, Mike
Darden, Oliver	Darnell, Rick
Daughtry, Mack
Davis, A. W.
Davis, Fred
Davis, Lee
Davis, Mickey
Davis, Mike
Davis, Warren
Davis, Willie
Dawson, Jim
DeAngelis, Billy
Dedmon, Lee
Dee, Don
DeHeer, Bill
Denton, Randy
DePre, Joe
Dickerson, Henry
Dickey, Clyde
Dickson, John
Dietrick, Coby
Dill, Craig
Dillard, Dwaine
Dinkins, Charles
Dinnel, Harry
Dinwiddie, Bill
Dixon, Al
Dodds, Stan
Dorsey, Ron
Dotson, Heyward
Dove, Sonny
Dover, Jerry
Driscoll, Terry
Duckworth, Larry
Dumas, Rich
Durham, Jarrett
Duval, Dennis
Dvoracek, Dennis
Dzik, Dick
E
Eakins, Jim
Early, Penny
Eatmon, Tony
Ebron,Roy
Edge, Charlie
Edmonds, Bobby Joe	Edwards, Robert
Edwards, Ozzie
Ellis, Chris
Ellis, Clarence
Ellis, Leroy
Elmore, Len
Elston, Darrell
England, Dan
English, Scott
Epps, Ed
Erickson, Rick
Erving, Julius	Evans, Billy
F
Faerber, George
Fagliano, Dick
Fairchild, John
Fields, Bobby	Finch, Larry
Fisher, Rick
Fitch, Jeff
Fitzhenry, John	Fleming, Ernie
Flournoy, Harry
Florence, Bobby
Flowers, Willie	Flynn, Mike
Ford, Bob
Ford, Gene
Forman, Matt
Foster, Jimmy
Fowler, Calvin
Franklin, Bill
Franz, Ron
Frazier, Wilbert
Freeman, Donnie	Fryer, Bernie
Fuller, Carl
Fultz, John
Fuqua, Richard
G
Gaines, Bill
Gaines, David
Galbo, Joe
Gale, Mike
Gallagher, Joe
Gammell, Curt	Gant, Matt
Gardner, Charles
Gardner, Ken
Garner, Bill
Garner, Rich
Gebler, Jack	Gerard, Gus
Gervin, George
Gillespie, Jack
Gilmore, Artis
Godfrey, Avery
Godfrey, John
Golden, Dave
Gords, Robert
Govan, Gerald
Graham, Cal
Grant, Hal
Grant, Travis
Gray, Gary
Greacen, Bobby
Green, James
Green, Lamar
Green, Luther	Green, Bill
Green, Mike
Green, Ron
Green, Steve
Greene, Jim
Greer, Lynn
Gregor, Gary
Gregory, Jim
Grey, Dennis
Grosso, Mike
Grubar, Dick
Guyette, Bob
H
Hackett, Rudy
Haderlein, Jim
Hadnot, Jim
Hagan, Cliff
Hagan, Tom
Hale, Hal
Halimon, Shaler
Halliburton, Jeff	Hamilton, Dennis
Hamilton, Joe
Hammond, Julian
Hamood, Joe
Hardin, Leonard
Harding, Reggie
Hardy, Darrell
Harge, Ira
Harkness, Jerry
Harris, Billy
Harris, Gordy
Harris, Pete
Harsher, Frank
Hauer, Mike
Hawkins, Connie
Hayes, Jim
Haywood, Spencer
Heath, Jon
Hentz, Charlie
Hess, Doug
Hester, Dan
Heyman, Art
Higgins, Billy
Higgins, Earle
Higgins, Lloyd
Hightower, Wayne
Hill, Seabern
Hill, Simmie
Hillman, Darnell
Hinton, Roy
Hogg, Dennis
Hogsett, Bob
Hollenbeck, Kent
Holliday, Larry
Holman, Denny
Holman, Larry
Holt, A.W.
Hooper, Bob
Hooser, Carroll
Hoover, Tom
Horn, Ron
Hoskins, Edward
Howard, Greg
Howard, Lavert	Hoxie, Randy
Hudson, Wendell
Hughes, Kim
Humes, Larry
Humes, Willie
Hummer, Ed
Hunter, Les
Hyder, Jerry
I
Ingelsby, Tom	Inniger, Irv	Irvine, George	Isaac, Warren	Issel, Dan	Iverson, Willie	Ivory, Elvin
J
Jabali, Warren
Jackson, Al
Jackson, Eddie
Jackson, George
Jackson, George
Jackson, Luke
Jackson, Merv
Jackson, Mike
Jackson, Tony
Jackson, Willie
James, Billy
Jankans, Billy
Janky, George
Jarvis, Jim
Jemison, Jesse
Jeter, Hal
Johnson, Ancil
Johnson, Ed
Johnson, George
Johnson, Gus
Johnson, Jerry
Johnson, Neil
Johnson, Pete
Johnson, Ralph
Johnson, Rich
Johnson, Stew
Johnson, Stewart
Jones, Bill
Jones, Billy
Jones, Bobby
Jones, Bubba
Jones, Caldwell
Jones, Collis
Jones, James
Jones, Johnny
Jones, Larry
Jones, Melvin
Jones, Nick
Jones, Rich
Jones, Steve
Jones, Wali
Jones,Wilbert 
Jordan, Charlie
Jorgenson, Neil
Joyce, Kevin
Joyner, Harry
K
Karl, George
Keller, Billy
Keller, Gary
Kelly, Arvesta
Kemp, Ed	Kendrick, Frank
Kennedy, Bill
Kennedy, Goo
Kennedy, Joe
Kennedy, Walker
Kenon, Larry
Kerr, Floyd
Kerwin, Jim
Keye, Julius
King, Isaiah
King, Jerry
King, Lloyd
King, Lamont
King, Ron
Kirkland, Wilbur
Kissane, Jim
Klutz, Lonnie
Knight, Billy
Knowles, Rod
Koehl, Larry
Kohls, Greg
Kondla, Tom
Koski, Tony
Kozlicki, Ron
Kramer, Barry
Kramer, Steve
Kron, Tommy
Kruse, Don
Kunze, Frank
Kunze, Terry
L
Lacefield, Reggie
Lacey, Edgar
Lackey, Bob
Ladner, Wendell
Lafayette, L.
Lamar, Bo
Lane, Clarence	Langston, David
LaRoche, Mike
Lattin, David
Laurie, Harry
Lawetzki, Jerry
Lawson, Al
Lawson, Ron
Layton, Mo
Leaks, Manny
Lee, Dave
Lee, Dick
Lee, Greg
Lee, Jim
Leedom, Bill
Leftwich, Ed
Lehman, George
Lentz, Leary
Lewis, Bob
Lewis, Earl
Lewis, Freddie
Lewis, Larry
Lewis, Mike	Lewis, Raymond
Liebowitz, Barry
Ligon, Jim
Lind, Fred
Lister, Jim
Little, Sam
Littles, Gene	Lloyd, Bob
Lloyd, Chuck
Lochman, Riney
Loescer, Gil
Logan, Henry
Long, Paul
Long, Willie
Lopes, Al	Love, Stan
Lowery, Charles
Lucas, Maurice
Lukacs, Frank
Lynam, R.B.
Lynch, Boyd
Lynn, Lonnie
Lynn, Mike
M
Mahaffey, Randy
Mahoney, Brian
Malone, Moses
Maloy, Mike
Manigault, Earl
Manning, Ed
Manning, Guy
Manning, Ted
Marshall, Hubie
Masiello, Tony
Mast, Eddie
Mathis, John
May, Ken
McCants, Sam
McClain, Ted
McDaniels, Jim
McDermott, Dennis
McDonald, Rod
McFarland, Pat
McGill, Bill
McGinnis, George
McGregor, Gil
McGriff, Elton
McGuire, Al
McHartley, Maurice
McIntyre, Bob
Rod McIntyre	McKee, Ed
McKee, Gerald
McLellan, Mike
McPipe, Roy
McReynolds, Thales
Melchionni, Bill
Menyard, Dewitt
Merriweather, Willie
Meyer, Bill
Meystedt, Kermit
Miasek, Dick
Miller, Jay
Miller, Larry
Miller, Tom
Mitchell, Leland
Mitchell, Peter
Mix, Steve
Mixon, Frank
Moe, Doug
Moffatt, Lloyd
Molis, Wayne
Moltimore, Lionel
Montgomery, Howie
Montgomery, Nap
Moore, Gene
Moore, Jimmy
Moore, Larry
Moore, Otto
Moore, Richie
Moreland, Jack
Morris, Billy
Morris, Larry
Morris, Larry
Morrison, John
Mount, Rick
Mueller, Erwin
Mullaney, Ed
Mundy, John
Murphy, Allen
Murrell, Willie
N
Napier, John
Nash, Bob
Nash, Cotton	Nater, Swen
Nau, Mike
Neary, James
Nelson, Garry
Nelson, Ron	Nemelka, Dick
Netolicky, Bob	Neumann, Johnny
Newmark, Dave
Newsom, Jerry	Newton, Bill
Newton, Joe
Niemann, Rich
Nolls, Randy
Nordholz, Mike
Norwood, Willie
Nowell, Mel
O
O'Brien, Jim
O'Brien, Jimmy	Obrovac, Dan
O'Conner, Rich	Ogden, Bud
O'Hanlon, Fred	Olberding, Mark
Olsen, Bud
O'Neal, Boyd	Orms, Barry
Overhouse, Jeff	Owens, Tom
P
Pack, Wayne
Padovan, George
Pagett, Dana
Palmer, Errol
Pangrazio, Joe
Parham, Greg
Parker, George
Parker, Joe
Parker, Rusty
Parkhill, Barry
Parks, Charley
Parks, Rich
Patterson, Jim
Patterson, Tom
Patton, Bob
Paulson, Eric
Paultz, Billy
Payne, Vernon
Peck, Tom
Peek, Richard
Peele, Rudy
Peeples, George
Pender, Jerry
Penix, Jim
Perry, Ron
Perry, Aulcie
Peterson, Mel
Pettis, Earnest
Pettit, Calvin
Pettway, Jerry
Petty, Otto
Phillips, Bryan
Phillips, Gene
Piatkowski, Walt
Pickens, Bill
Pickens, Clyde
Pitts, Jim
Plant, Letson
Plowden, Paul
Pondexter, Roscoe
Porter, Willie
Postley, John
Potter, Charles
Powell, Charlie
Powell, Cincy
Powers, Tim
Pradd, Marlbert
Pratt, Mike
Presley, Bob
Previs, Steve
Price, George
Price, Jovan
Price, Mike
Purkhieser, Bob
Q
Quick, Bob	 	 	 	 	 	 
R
Rackley, Luther
Rae, Don
Rambo, John
Ramsey, Jeff
Rascoe, Bobby
Rayl, Jimmy
Raymond, Craig
Reaves, Joe
Redd, Mike
Reed, Mel
Rhine, Kendall
Rhodes, Bob
Rhodes, Gary
Ribock, John
Richards, Doug
Richardson, Nolan
Richardson, Ron
Ricketts, Bob
Riedy, Bob
Rinaldi, Rich
Rinka, John
Ritter, John
Robbins, Red
Roberts, Joe
Roberts, Marv
Robertson, James
Robinson, Charlie
Robinson, Flynn
Robinson, Ronnie
Robinson, Sam
Robinson, Wil
Robisch, Dave
Roche, John
Rogers, Harry
Rogers, Willie
Rook, Jerry
Rooney, Mike
Ross, Bob
Roundfield, Dan
Roy, Tom
Royals, Reggie
Rozenski, Mike
Rue, Albi
Ruffner, Paul
Russell, Boonie
Russell, Pierre
Russell, Rubin
Ryan, John
S
Sadlier, Don
Salvadori, Al
Sanders, Al
Sanford, Ron
Sapit, Tony
Saulters, Glynn
Scaife, Don
Schaeffer, Billy
Schmidt, Rick
Schultz, Rich
Schurig, Roger
Scott, Charlie
Scott, Ray	Scott, Willie
Scranton, Paul
Sczerbiak, Walt
Seals, Bruce
Selvage, Les
Shackelford, Lynn
Sheffield, Larry
Shegogg, Cliff
Shepherd, Billy
Shepherd, Sam
Sherrod, Clarence
Sibley, Mark
Sidle, Don	Silas, James
Simmons, Grant
Simmons, Sam
Simon, Walt
Simpson, Ralph
Sims, Bob
Sims, Doug
Sims, Ernie
Singletary, Harry
Singletary, Sylvester
Skinner, Al
Skurcenski, Lou
Sloan, Bruce	Smith, Adrian
Smith, Al
Smith, Bill
Smith, Garfield
Smith, Howard
Smith, Jeff
Smith, Jim
Smith, John
Smith, John
Smith, Ken
Smith, Pete
Smith, Sam
Smothers, Bill	Sojourner, Willie
Somerset, Willie
Spain, Ken
Sparks, Dan
Spencer, George
Spight, Jolly
Spraggins, Bruce
Spruill, Allen
St. Pierre, Rob
Staak, Bob
Staggs, Erv
State, Jack	Standard, Frank
Stephens, Frank
Stephenson, Art
Stephenson, Jim
Steward, Abe
Stewart, Dennis
Stith, Tom
Stocks, Jim
Stoglin, Andy
Stoll, Randy
Stone, George
Stovall, Paul
Strauss, Ketchell
Strawder, Joe
Strickland, Bill
Stronczek, Frank
Stroud, Red
Styles, Tony
Suder, George
Summors, Glenn
Sutor, George
Sutter, John
Suttle, Rick
Swagerty, Keith
Swift, Skeeter
Szcerbiak, Walt
T
Tart, Lavern
Tatum, Cal
Taylor, Brian
Taylor, Les
Taylor, Ollie
Taylor.Roland
Taylor, Ron
Temple, Collis
Terry, Claude
Terry, Chuck	Thacker, Tom
Tharpe, Dan
Theard, Floyd
Thigpen, Justus
Thomas, Huey	Thomas, Ron
Thomas, Willis
Thompson, David
Thompson, George	Thompson, Jack
Thoren, Skip
Thornton, Dallas
Tinsley, George
Towe, Monte	Trgovich, Pete
Triplett, Ruben
Truitt, Ansley
Tschogl, John
Tucker, Al	Turner, Clyde
Turner, Gary
Turner, Herschel
Twardzik, Dave
U
 	 	 	 	 	 	 
V
Vacendak, Steve	Van Bloomestey, Tim	Van Breda Kolff, Jan
Van Schepen, Jim	Van Zant, Dennis
Vance, Tyree	Vaughn, Chico	Vaughn, Dave
Verga, Bob	Vilcheck, Al
Virden, Claude
W
Wagner, Phil
Walker, Cozel
Walker, Leroy
Walker, Rich
Walker, Steve
Wallace, Charles
Wallace, Joe
Wallace, Perry
Wallace, Richard
Waller, Dwight
Ward, Henry
Ward, Truman
Ware, Jim
Warhaftig, Jay
Warley, Ben
Warlick, Bob
Warren, Bob
Warren, Connie
Washington, Bobby
Washington, Donald
Washington, Tom
Watts, Sam
Waxman, Dave
Webb, Eldridge
Weber, Thorpe
Webster, Elnardo
Webster, Marvin
Wenzel, Bob
Werhas, Johnny
West, Gene
West, Lou
Westbrook, Dexter
White, Hubie
Whitehead, Don
Whitmore, Bob
Whitney, Hank
Widby, Ron
Wierman, Ted
Wilburn, Ken
Wiley, Gene
Williams, Al
Williams, Art
Williams, Bernie
Williams, Charlie
Williams, Chuck
Williams, David
Williams, Fly
Williams, Garland
Williams, Gene
Williams, Henry
Williams, Milt	Williams, Wayne
Williamson, John
Williford, Vann
Wilson, Bob
Wilson, Bobby
Wilson, C.D.
Wilson, Ed
Wilson, Isaiah
Wilson, Jasper
Wilson, Jimmy
Wilson, Sam
Wilson, Steve	Wimberley, Riley
Winkler, Marv
Wise, Skip
Wise, Willie
Witkowski, Dennis
Witt, Chuck
Wittman, Greg
Wolters, Willie
Wolthers, Danny
Woods, Tommy
Woolard, Bob
Workman, Tom
Worsley, Willie
Wright, Dave
Wright, Howard
Wright, Joby
Wright, Leroy
Wright, Lonnie
Wright, Sterling
Wuycik, Dennis
Wyatt, James
Wyatt, Levi
Wyenandt, Bo
xyz
Yates, Don	Yelverton, Charlie	Young, Arnold	Young, Kermit	Young, Skip	Zeller, Gary	Zoretich, Bob"""


In [252]:
"Card, Frank" in ABA

True

In [254]:
aba_list = []
for rname in rev_right_names:
    if rname not in ABA:
        aba_list.append(rname)

In [256]:
len(aba_list)

262

In [257]:
len(rev_right_names)

491

In [258]:
aba_list

['Abramovic, John',
 'Anderson, Andrew',
 'Aubuchon, Chet',
 'Bach, Johnny',
 'Baker, Norm',
 'Baltimore, Herschel',
 'Barnett, Nathaniel',
 'Barr, John',
 'Baumholtz, Frankie',
 'Becker, Arthur',
 'Becker, Moe',
 'Beenders, Hank',
 'Bergh, Larry',
 'Bialosuknia, Wesley',
 'Biasatti, Hank',
 'Bishop, Gale',
 'Bloom, Mike',
 'Booker, Butch',
 'Bowens, Tommie',
 'Brightman, Al',
 'Brindley, Aud',
 'Brown, Darrell',
 'Brown, Harold',
 'Brown, Leon',
 'Bustion, David',
 'Byrd, Walter',
 'Bytzura, Michael',
 'Callahan, Tom',
 'Calverley, Ernie',
 'Carlisle, Chet',
 'Chubin, Stephen',
 'Clark, Richard',
 'Cluggish, Bob',
 'Colone, Joe',
 'Congdon, Jeffrey',
 'Connors, Chuck',
 'Corley, Ken',
 'Crisler, Hal',
 'Critchfield, Russell',
 'Cure, Armand',
 'Dallmar, Howie',
 'Darden, Ollie',
 'Davis, Aubrey',
 'Davis, Bill',
 'Dawson, Jimmy',
 'Dehnert, Red',
 'Dille, Bob',
 'Downey, Bill',
 'Dwan, Jack',
 'Ebron, Roy',
 'Edge, Charles',
 'Eggleston, Lonnie',
 'Ehlers, Bulbs',
 'Eliason, Don',
 'E